In [ ]:
# Zach Fizet
# CS-340
# Project Two

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import os
import pandas as pd

# Import your custom MongoDB CRUD module
from animal_shelter import AnimalShelter

# MongoDB Authentication
username = "aacuser"
password = "12345"
host = "nv-desktop-services.apporto.com"
port = 32738

db = AnimalShelter(username, password, host=host, port=port, db_name="AAC", collection_name="animals")

# Retrieve all animal records for initial load
df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Load and encode logo
image_filename = 'grazioso-logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Start Dash app
app = JupyterDash(__name__)

# Layout
app.layout = html.Div([
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'100px'}),
        html.A("Visit Grazioso Salvare", href='https://www.snhu.edu', target="_blank"),
        html.P("Dashboard developed by Zachary Fizet", style={'fontWeight': 'bold'}),
    ], style={'textAlign': 'center'}),

    html.Center(html.H1('Grazioso Salvare Animal Dashboard')),
    html.Hr(),

    # Filter radio buttons
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block'}
    ),

    html.Hr(),

    # DataTable for displaying records
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
        filter_action="native",
        sort_action="native",
        row_selectable="single"
    ),

    html.Br(),
    html.Hr(),

    # Form for Adding / Editing Animal Records
    html.Div([
        html.H3("Add or Edit Animal Record"),
        html.Div([
            html.Label("Name"),
            dcc.Input(id='input-name', type='text', placeholder='Enter animal name'),
        ]),
        html.Div([
            html.Label("Breed"),
            dcc.Input(id='input-breed', type='text', placeholder='Enter breed'),
        ]),
        html.Div([
            html.Label("Sex Upon Outcome"),
            dcc.Input(id='input-sex', type='text', placeholder='Enter sex upon outcome'),
        ]),
        html.Div([
            html.Label("Age Upon Outcome in Weeks"),
            dcc.Input(id='input-age', type='number', placeholder='Enter age in weeks'),
        ]),
        html.Div([
            html.Label("Location Latitude"),
            dcc.Input(id='input-lat', type='number', placeholder='Enter latitude'),
        ]),
        html.Div([
            html.Label("Location Longitude"),
            dcc.Input(id='input-long', type='number', placeholder='Enter longitude'),
        ]),
        html.Br(),
        html.Button('Add New Record', id='btn-add', n_clicks=0),
        html.Button('Update Selected Record', id='btn-update', n_clicks=0),
        html.Button('Delete Selected Record', id='btn-delete', n_clicks=0),
        html.Div(id='action-status', style={'marginTop': '10px', 'fontWeight': 'bold'})
    ], style={'border': '1px solid #ccc', 'padding': '20px', 'borderRadius': '5px', 'maxWidth': '400px'}),

    html.Br(),
    html.Hr(),

    # Charts side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#########################################
# Callbacks
#########################################

# Filtering logic
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$lte": 104}
        }
    elif filter_type == 'mountain':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$lte": 104}
        }
    elif filter_type == 'disaster':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$lte": 104}
        }
    else:
        query = {}

    results = pd.DataFrame.from_records(db.read(query))
    if '_id' in results.columns:
        results.drop(columns=['_id'], inplace=True)
    return results.to_dict('records')

# Update pie chart based on current table view
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        dff = df
    else:
        dff = pd.DataFrame(viewData)

    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Breed Distribution')
        )
    ]

# Highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
            dl.Tooltip(dff.iloc[row]['breed']),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row]['name'])
            ])
        ])
    ])]

# ---------------------------------------------
# New callback to populate input fields when row selected for editing
@app.callback(
    [Output('input-name', 'value'),
     Output('input-breed', 'value'),
     Output('input-sex', 'value'),
     Output('input-age', 'value'),
     Output('input-lat', 'value'),
     Output('input-long', 'value')],
    [Input('datatable-id', 'derived_virtual_selected_rows'),
     Input('datatable-id', 'derived_virtual_data')]
)
def populate_inputs(selected_rows, data):
    if selected_rows is None or len(selected_rows) == 0:
        # No selection, clear inputs
        return "", "", "", None, None, None
    else:
        row = selected_rows[0]
        dff = pd.DataFrame(data)
        # Fill inputs with selected row's data
        return (
            dff.iloc[row].get('name', ''),
            dff.iloc[row].get('breed', ''),
            dff.iloc[row].get('sex_upon_outcome', ''),
            dff.iloc[row].get('age_upon_outcome_in_weeks', None),
            dff.iloc[row].get('location_lat', None),
            dff.iloc[row].get('location_long', None),
        )

# ---------------------------------------------
# Callback for Add, Update, Delete buttons
@app.callback(
    Output('action-status', 'children'),
    Output('datatable-id', 'data'),
    [Input('btn-add', 'n_clicks'),
     Input('btn-update', 'n_clicks'),
     Input('btn-delete', 'n_clicks')],
    [State('input-name', 'value'),
     State('input-breed', 'value'),
     State('input-sex', 'value'),
     State('input-age', 'value'),
     State('input-lat', 'value'),
     State('input-long', 'value'),
     State('datatable-id', 'derived_virtual_selected_rows'),
     State('datatable-id', 'derived_virtual_data')]
)
def handle_crud(btn_add, btn_update, btn_delete, name, breed, sex, age, lat, long, selected_rows, data):
    ctx = dash.callback_context

    if not ctx.triggered:
        return "", data

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # Reload current data from DB to ensure consistency after any operation
    current_df = pd.DataFrame.from_records(db.read({}))
    if '_id' in current_df.columns:
        current_df.drop(columns=['_id'], inplace=True)

    # Prepare the document from form inputs
    document = {
        "name": name,
        "breed": breed,
        "sex_upon_outcome": sex,
        "age_upon_outcome_in_weeks": int(age) if age is not None and age != '' else None,
        "location_lat": float(lat) if lat is not None and lat != '' else None,
        "location_long": float(long) if long is not None and long != '' else None
    }

    # Remove keys with None values (MongoDB does not like None for some fields)
    document = {k: v for k, v in document.items() if v is not None and v != ""}

    if button_id == 'btn-add':
        # Insert new record
        success = db.create(document)
        if success:
            message = "New record added successfully."
        else:
            message = "Failed to add new record."

    elif button_id == 'btn-update':
        # Must have a selected row to update
        if selected_rows is None or len(selected_rows) == 0:
            return "Please select a row to update.", data
        else:
            selected_row = selected_rows[0]
            dff = pd.DataFrame(data)
            # Use unique criteria to find the record in DB (here, name + breed + sex + lat + long assumed unique)
            # Adjust if your dataset has a unique ID field
            query = {
                "name": dff.iloc[selected_row]['name'],
                "breed": dff.iloc[selected_row]['breed'],
                "sex_upon_outcome": dff.iloc[selected_row]['sex_upon_outcome'],
                "location_lat": dff.iloc[selected_row]['location_lat'],
                "location_long": dff.iloc[selected_row]['location_long']
            }
            modified = db.update_one(query, document)
            if modified:
                message = "Record updated successfully."
            else:
                message = "Failed to update record."

    elif button_id == 'btn-delete':
        # Must have a selected row to delete
        if selected_rows is None or len(selected_rows) == 0:
            return "Please select a row to delete.", data
        else:
            selected_row = selected_rows[0]
            dff = pd.DataFrame(data)
            query = {
                "name": dff.iloc[selected_row]['name'],
                "breed": dff.iloc[selected_row]['breed'],
                "sex_upon_outcome": dff.iloc[selected_row]['sex_upon_outcome'],
                "location_lat": dff.iloc[selected_row]['location_lat'],
                "location_long": dff.iloc[selected_row]['location_long']
            }
            deleted = db.delete_one(query)
            if deleted:
                message = "Record deleted successfully."
            else:
                message = "Failed to delete record."

    else:
        return "", data

    # Reload updated data after any CRUD operation
    new_df = pd.DataFrame.from_records(db.read({}))
    if '_id' in new_df.columns:
        new_df.drop(columns=['_id'], inplace=True)

    return message, new_df.to_dict('records')

# Run app
app.run_server(debug=True, mode='inline')


MongoDB connection successful.


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

